# Introduction to Tensors, TensorFlow Functions and TensorFlow Datasets

In this tutorial, we will learn about some key aspects of TensorFlow. We will start by discussing tensors, then we will talk about `tf.function`s and when to use them. Lastly we will discuss `tf.data.Dataset` methods and how to create tensor-formatted datasets.

In [1]:
import tensorflow as tf
import numpy as np

## Basics of tensors

Let's create some basic tensors.

A tensor is a multi-dimensional array with a consistent type (known as a `dtype`).  All supported `dtypes` cna be seen with `tf.dtypes.DType`.

Tensors are similar to [NumPy](https://numpy.org/devdocs/user/quickstart.html)  `np.arrays`.

Tensors are immutable, just like Python numbers and strings. The contents of a tensor can never be updated; only new ones can be created.


A "scalar" is a "rank-0" tensor. It contains a single value and no "axes".

In [ ]:
# This will be an int32 tensor by default; see "dtypes" below.
rank_0_tensor = tf.constant(4)
print(rank_0_tensor)

A "vector" is a "rank-1" tensor and contains something like a list of values. A vector is characterized by having one axis.

In [ ]:
# Let's make this a float tensor.
rank_1_tensor = tf.constant([2.0, 3.0, 4.0])
print(rank_1_tensor)

A "matrix" is a "rank-2" tensor and it has two axes. This is what a single channel image turns out to be in tensor format.

In [ ]:
# If you want to be specific, you can set the dtype (see below) at creation time
rank_2_tensor = tf.constant([[1, 2],
                             [3, 4],
                             [5, 6]], dtype=tf.float16)
print(rank_2_tensor)

<table>
<tr>
  <th>A scalar, shape: <code>[]</code></th>
  <th>A vector, shape: <code>[3]</code></th>
  <th>A matrix, shape: <code>[3, 2]</code></th>
</tr>
<tr>
  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/scalar.png?raw=1" alt="A scalar, the number 4" />
  </td>

  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/vector.png?raw=1" alt="The line with 3 sections, each one containing a number."/>
  </td>
  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/matrix.png?raw=1" alt="A 3x2 grid, with each cell containing a number.">
  </td>
</tr>
</table>


Tensors can feature more axes, as is required for representation of multi-channel images. For example, we can have a tensor with three axes.

In [ ]:
# There can be an arbitrary number of
# axes (sometimes called "dimensions")
rank_3_tensor = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])

print(rank_3_tensor)

A tensor with more than two axes can be visualized in several ways.

<table>
<tr>
  <th colspan=3>A 3-axis tensor, shape: <code>[3, 2, 5]</code></th>
<tr>
<tr>
  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/3-axis_numpy.png?raw=1"/>
  </td>
  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/3-axis_front.png?raw=1"/>
  </td>

  <td>
   <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/3-axis_block.png?raw=1"/>
  </td>
</tr>

</table>

NumPy arrays can be created from tensors using either the `np.array` or `tensor.numpy` methods:

In [ ]:
np.array(rank_2_tensor)

In [ ]:
rank_2_tensor.numpy()

Most tensors we encounter in computer vision contain floats and integers, but tensors can also represent other types, such as:

* complex numbers
* strings

TensorFlow requires a `tf.Tensor` to be "rectangular," which means that every element along an axis is the same size. There are exceptions to this (ragged and sparse tensors, but those are more rare and mostly out of scope for now).

We can calculate basic mathematical operations on tensors, such as addition, element-wise multiplication, and matrix multiplication.

In [ ]:
a = tf.constant([[1, 2],
                 [3, 4]])
b = tf.constant([[1, 1],
                 [1, 1]]) # Could have also said `tf.ones([2,2], dtype=tf.int32)`

print(tf.add(a, b), "\n") # element-wise addition
print(tf.multiply(a, b), "\n") # element-wise multiplication
print(tf.matmul(a, b), "\n") # matrix multiplication

In [ ]:
print(a + b, "\n") # element-wise addition
print(a * b, "\n") # element-wise multiplication
print(a @ b, "\n") # matrix multiplication

Tensors are used in many types of operations (or "Ops"), such as common machine learning operations like `tf.math.argmax` and `tf.nn.softmax`.

In [ ]:
c = tf.constant([[4.0, 5.0], [10.0, 1.0]])

# Find the largest value
print(tf.reduce_max(c))
# Find the index of the largest value
print(tf.math.argmax(c))
# Compute the softmax
print(tf.nn.softmax(c))

Generally, where tensors are expected, TensorFlow functions additionally accept anything that can be converted to a `Tensor` using `tf.convert_to_tensor`.

In [ ]:
tf.convert_to_tensor([1,2,3]) # List

In [ ]:
tf.reduce_max([1,2,3]) # List

In [ ]:
tf.reduce_max(np.array([1,2,3])) # NumPy array

## Significance of Tensor shapes

A tensor has a shape.  Some key terms to know:

* **Shape**: Describes the number of axes and length (number of elements) in each axis of a tensor.
* **Rank**: Determined by the number of axes in a tensor.  If you recall, scalars are rank 0, vectors rank 1, matrices rank 2.
* **Axis** or **Dimension**: A dimension of a tensor.
* **Size**: The total number of elements in a tensor. This can be expressed as the product of the shape vector's elements, e.g. a tensor shape of (256, 256, 3) will have a size of 196608.


`tf.TensorShape` objects describe these key properties and more.

In [3]:
rank_4_tensor = tf.zeros([3, 2, 4, 5])

<table>
<tr>
  <th colspan=2>A rank-4 tensor, shape: <code>[3, 2, 4, 5]</code></th>
</tr>
<tr>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/shape.png?raw=1" alt="A tensor shape is like a vector.">
    <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/4-axis_block.png?raw=1" alt="A 4-axis tensor">
  </td>
  </tr>
</table>


In [ ]:
print("Type of every element:", rank_4_tensor.dtype)
print("Number of axes:", rank_4_tensor.ndim)
print("Rank: ", tf.rank(rank_4_tensor))
print("Shape of tensor:", rank_4_tensor.shape)
print("Elements along axis 0 of tensor:", rank_4_tensor.shape[0])
print("Elements along the last axis of tensor:", rank_4_tensor.shape[-1])
print("Total number of elements (3*2*4*5): ", tf.size(rank_4_tensor).numpy())

Axes of a tensor follow a global to local ordering. For example, in machine learning we often talk about batches. For a given batch, the dimension of the batch is ordered first, followed by the batch element's spatial dimensions (height, width) and then the channels (referred to as features in the below diagram).

<table>
<tr>
<th>Typical axis order</th>
</tr>
<tr>
    <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/shape2.png?raw=1" alt="Keep track of what each axis is. A 4-axis tensor might be: Batch, Width, Height, Channels">
  </td>
</tr>
</table>

## Indexing

### Single-axis indexing

In TensorFlow, standard Python indexing rules apply. Indexing tensors looks similar to [indexing a list in Python](https://docs.python.org/3/tutorial/introduction.html#strings). The same applied for NumPy-style indexing.

* indexes begin at `0`
* negative indices count backwards from the end
* colons, `:`, are used for slices: `start:stop:step`


In [ ]:
rank_1_tensor = tf.constant([0, 1, 1, 2, 3, 5, 8, 13, 21, 34])
print(rank_1_tensor.numpy())

When using a scalar to get an element in the tensor, the axis is removed.

In [ ]:
print("First:", rank_1_tensor[0].numpy())
print("Second:", rank_1_tensor[1].numpy())
print("Last:", rank_1_tensor[-1].numpy())

When a slice is indexed using `:` the axis is retained:

In [ ]:
print("Everything:", rank_1_tensor[:].numpy())
print("Before 4:", rank_1_tensor[:4].numpy())
print("From 4 to the end:", rank_1_tensor[4:].numpy())
print("From 2, before 7:", rank_1_tensor[2:7].numpy())
print("Every other item:", rank_1_tensor[::2].numpy())
print("Reversed:", rank_1_tensor[::-1].numpy())

### Multi-axis indexing

When working with higher ranks of tensors, we may have to use multiple indices.

To do this, we treat each axis independently with the exact same rules as in the single-axis case.

In [ ]:
print(rank_2_tensor.numpy())

If we pass an integer for each index for each axis, a scalar is returned.

In [ ]:
# Pull out a single value from a 2-rank tensor
print(rank_2_tensor[1, 1].numpy())

We can combine integers and slices when indexing too. This can be useful if, for example, we want a subset of an image.

In [ ]:
# Get row and column tensors
print("Second row:", rank_2_tensor[1, :].numpy())
print("Second column:", rank_2_tensor[:, 1].numpy())
print("Last row:", rank_2_tensor[-1, :].numpy())
print("First item in last column:", rank_2_tensor[0, -1].numpy())
print("Skip the first row:")
print(rank_2_tensor[1:, :].numpy(), "\n")

An example for a tensor with 3 axes:

In [ ]:
print(rank_3_tensor[:, :, 4])

<table>
<tr>
<th colspan=2>Selecting the last feature across all locations in each example in the batch </th>
</tr>
<tr>
    <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/index1.png?raw=1" alt="A 3x2x5 tensor with all the values at the index-4 of the last axis selected.">
  </td>
      <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/index2.png?raw=1" alt="The selected values packed into a 2-axis tensor.">
  </td>
</tr>
</table>

More on how to apply indexing can be found in the [tensor slicing guide](https://tensorflow.org/guide/tensor_slicing).

## Manipulating tensor shapes

Sometimes, we need to reshape a tensor. For example, we flatten rank 2 tensors to rank 1 when computing things like confusion matrices.


In [ ]:
# Shape returns a `TensorShape` object that shows the size along each axis
x = tf.constant([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])
print(x.shape)

In [12]:
# Flatten the rank 2 tensor to rank 1 (a vector)
x_flat = tf.reshape(x, [-1])
print(x_flat)

In [ ]:
# Shape returns a `TensorShape` object that shows the size along each axis
x = tf.constant([[1], [2], [3]])
print(x.shape)

In [ ]:
# You can convert this object into a Python list, too
print(x.shape.as_list())

A tensor can be manipulated into a new shape using the `tf.reshape` operation.

In [ ]:
# You can reshape a tensor to a new shape.
# Note that you're passing in a list
reshaped = tf.reshape(x, [1, 3])

In [ ]:
print(x.shape)
print(reshaped.shape)

The original tensor and the newly created one are both held in memory (remember tensors are immutable). TensorFlow abides by C-style "row-major" memory ordering, which means that an increment on the rightmost index corresponds to a single step in memory.

In [ ]:
print(rank_3_tensor)

In fact, flattening a tensor shows the order in which it is held in memory.

In [ ]:
# A `-1` passed in the `shape` argument says "Whatever fits".
print(tf.reshape(rank_3_tensor, [-1]))

Generally we use `tf.reshape` to combine or split adjacent axes (or add/remove `1`s, similar to `np.squeeze`).

Taking the below 3x2x5 tensor, we can reshape to (3x2)x5 or 3x(2x5) for example.

In [ ]:
print(tf.reshape(rank_3_tensor, [3*2, 5]), "\n")
print(tf.reshape(rank_3_tensor, [3, -1]))

<table>
<th colspan=3>
Some good reshapes.
</th>
<tr>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-before.png?raw=1" alt="A 3x2x5 tensor">
  </td>
  <td>
  <img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-good1.png?raw=1" alt="The same data reshaped to (3x2)x5">
  </td>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/reshape-good2.png?raw=1" alt="The same data reshaped to 3x(2x5)">
  </td>
</tr>
</table>


Notes:
*   We can reshape a tensor to a new shape so long as it entails the same total number of elements (size).
*   Shifting the order of axes is done with `tf.transpose` instead of `tf.reshape`.
*   You may run across not-fully-specified shapes. Either the shape contains a `None` (an axis-length is unknown) or the whole shape is `None` (the rank of the tensor is unknown). This is often useful for machine learning when the images in a batch may have non-uniform width and height dimensions.




## Tensor `DTypes`

We can find a `tf.Tensor`'s data type by inspecting the `Tensor.dtype` property.

Datatypes can be optionally specified when creating a `tf.Tensor` from a Python object. If left unspecified, TensorFlow inuits and assigns a datatype that can reasonably represent the data. By default, TensorFlow translates Python integers as `tf.int32` and Python floating point numbers as `tf.float32`.

Tensors can be cast to other types.

In [ ]:
the_f64_tensor = tf.constant([2.2, 3.3, 4.4], dtype=tf.float64)
the_f16_tensor = tf.cast(the_f64_tensor, dtype=tf.float16)
# Now, cast to an uint8 and lose the decimal precision
the_u8_tensor = tf.cast(the_f16_tensor, dtype=tf.uint8)
print(the_u8_tensor)

## Broadcasting

Broadcasting in TensorFlow is based on the [equivalent method in NumPy](https://numpy.org/doc/stable/user/basics.broadcasting.html).  The premise is that sometimes we need to stretch smaller tensors automatically to fit the size of larger tensors to run combined operations on them.

A simple example is multiplying or adding a tensor to a scalar. The scalar in this case is broadcast to the same shape as the tensor.

In [ ]:
x = tf.constant([1, 2, 3])

y = tf.constant(2)
z = tf.constant([2, 2, 2])
# All of these are the same computation
print(tf.multiply(x, 2))
print(x * y)
print(x * z)

Furthermore, tensors with an axis of length 1 can be stretched to match the axis length of another tensor during a combined operation.  

As an example, a 3x1 matrix can be element-wise multiplied by a 1x4 matrix to result in a 3x4 matrix. The notation of the axis with length 1 is optional. In other words, the real shape of y is `[4]`.

In [ ]:
# These are the same computations
x = tf.reshape(x,[3,1])
y = tf.range(1, 5)
print(x, "\n")
print(y, "\n")
print(tf.multiply(x, y))

<table>
<tr>
  <th>A broadcasted add: a <code>[3, 1]</code> times a <code>[1, 4]</code> gives a <code>[3,4]</code> </th>
</tr>
<tr>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/broadcasting.png?raw=1" alt="Adding a 3x1 matrix to a 4x1 matrix results in a 3x4 matrix">
  </td>
</tr>
</table>


The same operation without broadcasting looks like this:

In [ ]:
x_stretch = tf.constant([[1, 1, 1, 1],
                         [2, 2, 2, 2],
                         [3, 3, 3, 3]])

y_stretch = tf.constant([[1, 2, 3, 4],
                         [1, 2, 3, 4],
                         [1, 2, 3, 4]])

print(x_stretch * y_stretch)  # Again, operator overloading

Broadcasting is usually both time and space efficient, as it never creates the expanded tensors in memory.  

The broadcasting operation can be observed with `tf.broadcast_to`.

In [ ]:
print(tf.broadcast_to(tf.constant([1, 2, 3]), [3, 3]))

More on broadcasting can be found in [this section](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html) of Jake VanderPlas's book _Python Data Science Handbook_.

## tf.convert_to_tensor

As a reminder, most TensorFlow operations that expect arguments of class `tf.Tensor`, such as `tf.matmul` and `tf.reshape`, will also accept Python objects shaped like tensors.

Under the hood, most TensorFlow ops apply `convert_to_tensor` to non-tensor arguments, such as NumPy's `ndarray`, `TensorShape`, Python lists, and `tf.Variable`, all of which will convert automatically.

See the conversion registry `tf.register_tensor_conversion_function` to know whether a data type will automatically convert.

## String tensors

`tf.string` is a `dtype` that represents data as strings (variable-length byte arrays) in tensors.

However, string tensors are atomic meaning that they cannot be indexed the way Python strings can. See `tf.strings` for ways to manipulate them.

An example of a scalar string tensor:

In [ ]:
# Tensors can be strings, too here is a scalar string.
scalar_string_tensor = tf.constant("Lima Peru")
print(scalar_string_tensor)

In [ ]:
# If you have three string tensors of different lengths, this is OK.
tensor_of_strings = tf.constant(["Lima Peru",
                                 "NASA SERVIR",
                                 "Development Seed"])
# Note that the shape is (3,). The string length is not included.
print(tensor_of_strings)

The `b` prefix in the above printout indicates that the `tf.string` dtype is a byte-string, not a unicode string. For more on how to use unicode text in TensorFlow see the [Unicode Tutorial](https://www.tensorflow.org/tutorials/load_data/unicode).

Also worth noting, unicode characters in TensorFlow are utf-8 encoded.

`tf.strings` contains some basic string operations, including `tf.strings.split`.

In [ ]:
# You can use split to split a string into a set of tensors
print(tf.strings.split(scalar_string_tensor, sep=" "))

In [ ]:
# ...but it turns into a what is known as a `RaggedTensor` if you split up a tensor of strings,
# as each string might be split into a different number of parts.
print(tf.strings.split(tensor_of_strings))

And we also have `tf.string.to_number`:

In [ ]:
text = tf.constant("1 10 100")
print(tf.strings.to_number(tf.strings.split(text, " ")))

In TensorFlow, the `tf.string` dtype describes all raw bytes data. The `tf.io` module is used to convert data to and from bytes, as is done when decoding images and parsing text.

## Sparse tensors

Data can sometimes be sparse, as might be the case for a wide embedding space.  TensorFlow has a datat type called `tf.sparse.SparseTensor` and related operations to support sparse data and work efficiently with it.

<table>
<tr>
  <th>A `tf.SparseTensor`, shape: <code>[3, 4]</code></th>
</tr>
<tr>
  <td>
<img src="https://github.com/tensorflow/docs/blob/master/site/en/guide/images/tensor/sparse.png?raw=1" alt="An 3x4 grid, with values in only two of the cells.">
  </td>
</tr>
</table>

In [ ]:
# Sparse tensors store values by index in a memory-efficient manner
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]],
                                       values=[1, 2],
                                       dense_shape=[3, 4])
print(sparse_tensor, "\n")

# You can convert sparse tensors to dense
print(tf.sparse.to_dense(sparse_tensor))

# Functions

In TensorFlow 2, [eager execution](eager.ipynb) mode is on by default. This mode enables intuitive use, rapid experimentation and flexibility. However, this mode faces limits with performance and deployability.

`tf.function` is used to make graphs out of programs. The critical case for it is when we need to build Python-independent dataflow graphs from Python code, with goal of creating performant and portable models. In fact, compiling in `tf.function` is required to use the `SavedModel` class.

We will discuss some of the main uses for `tf.function`.

Before that, it's worth knowing how and when `tf.function` is recommended:

- Debugging is best done in eager mode, then functions from that should be decorated with `@tf.function`.
- Use `tf.function` to avoid on Python side effects like object mutation or list appends.

### Usage

A `Function` in TensorFlow, (which can be created by adding the `@tf.function` decorator) behaves like a core TensorFlow operation. It can be executed eagerly.

In [ ]:
@tf.function  # The decorator converts `add` into a `Function`.
def add(a, b):
  return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

`Function`s can be used inside other `Function`s.

In [ ]:
@tf.function
def dense_layer(x, w, b):
  return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

In some cases, `Function`s are faster than their eager code equivalent. This is especially so for graphs with lots of small ops. However, on the flip side graphs with few expensive ops (like convolutions) may not benefit from much speedup.

In [ ]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# Warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")

#### Use unknown dimensions for flexibility

  TensorFlow matches tensorson the basis of their shape. We can use a `None` dimension as a wildcard to allow `Function`s to run on variably-sized input. Variably-sized input is frequent when dealing with sequences of different length, or images of different spatial dimensions (sizes) across batches.

In [ ]:
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def g(x):
  print('Tracing with', x)
  return x

# No retrace!
print(g(tf.constant([1, 2, 3])))
print(g(tf.constant([1, 2, 3, 4, 5])))

#### Using iterators and generators

TensorFlow has a `tf.data.Iterator` for iteration. The [`tf.data`](https://www.tensorflow.org/guide/data) API helps implement generator patterns. TensorFlow's iterators and generators help avoid Python side effects which can surface outside of eager mode.

In [ ]:
@tf.function
def buggy_consume_next(iterator):
  tf.print("Value:", next(iterator))

iterator = iter([1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)


In [ ]:
@tf.function
def good_consume_next(iterator):
  # This is ok, iterator is a tf.data.Iterator
  tf.print("Value:", next(iterator))

ds = tf.data.Dataset.from_tensor_slices([1, 2, 3])
iterator = iter(ds)
good_consume_next(iterator)
good_consume_next(iterator)
good_consume_next(iterator)

# Data

The `tf.data.Dataset` [API](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) represents methods to create tensor-formatted datasets out of many different types of data. It's also used in the making of data pipelines.

Usage of the API follows the below pattern:

*   Create a dataset from your input data.
*   Iterate over the dataset, applying any dataset transformations to the data.

Note that iteration occurs through streaming, so it dones't matter if a full dataset can't fit into memory.

The simplest example of creating a dataset is from a python list:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset:
  print(element)

Transformations:
With a dataset, you can use transformations to prepare the data. In machine learning, sometimes we want to augment our data with image flips and rotations for example. Those could be done in the pipeline too.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
dataset = dataset.map(lambda x: x*2)
list(dataset.as_numpy_iterator())

Common Terms:


*   **Element**: A single result from calling next() on a dataset iterator. Elements are sometimes nested structures containing multiple components.

*   **Component**: The single part in a nested structure of an element.

Supported types:

As nested structures, elements can contain tuples, named tuples, and dictionaries. This is different behavior than that of Python lists. Instead, Python lists require conversion to tensors and then may be treated as components. Element components can express as any type represented in `tf.TypeSpec`, which includes `tf.Tensor`, `tf.data.Datase`t, `tf.sparse.SparseTensor`, and `tf.TensorArray`.

In [ ]:
a = 1 # Integer element
b = 2.0 # Float element
c = (1, 2) # Tuple element with 2 components
d = {"a": (2, 2), "b": 3} # Dict element with 3 components
Point = collections.namedtuple("Point", ["x", "y"])
e = Point(1, 2) # Named tuple
f = tf.data.Dataset.range(10) # Dataset element

This discussion on datasets is different than the [TensorFlow Datasets collection](https://www.tensorflow.org/datasets) which hosts prepared datasets for TensorFlow models and analysis. We will explore use of the TensorFlow Datasets collection and more on the `tf.data.Dataset` API iterators and generators (datat pipes) in a follow up lesson.

This notebook was adapted from [Introduction to Tensors](https://www.tensorflow.org/guide/tensor), [TensorFlow Functions](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/function.ipynb#scrollTo=HQrG5_kOiKl_), and [tf.data.Dataset API](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).